In [2]:
import os
import whisper
import ffmpeg  # replaced moviepy
from langchain_community.llms import Ollama

# Load models
whisper_model = whisper.load_model("base")   # Use "small", "medium" for better accuracy if needed
llm = Ollama(model="llama3.2")          # Ensure Ollama with llama3 is running

# Extract audio from video using ffmpeg-python
def extract_audio(video_path, output_audio="audio2.wav"):
    (
        ffmpeg
        .input(video_path)
        .output(output_audio, format='wav', acodec='pcm_s16le', ac=1, ar='16000')
        .run(overwrite_output=True)
    )
    return output_audio

# Transcribe audio to text
def transcribe(audio_path):
    result = whisper_model.transcribe(audio_path)
    return result["text"]

# Summarize using LLM
def summarize(transcript):
    prompt = f"""
You are an expert AI assistant reviewing an educational video on Generative AI. Given the transcript below, generate a well-structured review in **Markdown format**.

Your response must include:

#  Video Summary
Provide a clear and concise summary of the video's content.

##  Topics Covered
List all major topics or subtopics discussed in the video.

##  Content Quality Evaluation
State whether the content is **Good**, **Moderate**, or **Poor**. Justify your answer with clear reasoning (clarity, flow, explanation depth, etc.).

##  Beginner-Friendliness
Analyze if this video is beginner-friendly. Are technical terms explained? Is it suitable for someone new to Generative AI?

##  Suggestions for Improvement
Suggest what more could be included to improve the video for beginners — missing concepts, better examples, visual aids, or practical demos.

##  Final Verdict
Provide a short conclusion on the video’s usefulness, quality, and whether it’s recommended for a Gen AI course.

---
Transcript:
\"\"\"
{transcript}
\"\"\"
"""

    return llm.invoke(prompt)

# Process single video
def summarize_video(video_path):
    print(f" Processing video: {video_path}")

    audio_path = extract_audio(video_path)
    transcript = transcribe(audio_path)
    markdown = summarize(transcript)

    os.remove(audio_path)  # Clean up

    output_file = os.path.splitext(os.path.basename(video_path))[0] + ".md"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(markdown)

    print(f" Markdown saved: {output_file}")

# Example usage
if __name__ == "__main__":
    video_path ="C:/Vedios_to_summarize/1.3. Types of AI.mp4"
    summarize_video(video_path)

 Processing video: C:/Vedios_to_summarize/1.3. Types of AI.mp4


c:\Users\Mastan\anaconda3\envs\video_summary\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Markdown saved: 1.3. Types of AI.md
